# Chatbot with Agents and Memory

## Imports

In [1]:
# System
import os
os.environ['USER_AGENT'] = 'JimYin88'
import requests
from datetime import date
from datetime import datetime
import math

# Geo Mapping
import geocoder
from geopy.geocoders import Nominatim

# LLM Models
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM, ChatOllama

# Templates
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import MessagesPlaceholder
from langchain import hub

# Document Loaders
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import WebBaseLoader
# from langchain.document_loaders import BSHTMLLoader
# from langchain_community.document_loaders import UnstructuredRTFLoader

# Document Splitters
from langchain.text_splitter import TokenTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_openai import OpenAIEmbeddings

# Vector Stores
from langchain_chroma import Chroma
from langchain_community.vectorstores.faiss import FAISS

# LangChain Chains
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio
import gradio as gr

# Tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.tools import Tool
from langchain.agents import tool, load_tools, initialize_agent, AgentType
from langchain_community.tools.google_books import GoogleBooksQueryRun
from langchain_community.utilities.google_books import GoogleBooksAPIWrapper
from langchain_community.tools.google_trends import GoogleTrendsQueryRun
from langchain_community.utilities.google_trends import GoogleTrendsAPIWrapper
from langchain_google_community import GooglePlacesTool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# from gradio_tools.tools import (
#     ImageCaptioningTool,
#     StableDiffusionPromptGeneratorTool,
#     StableDiffusionTool,
#     TextToVideoTool,
# )

# History Memory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Text-to-Speech
import elevenlabs
from elevenlabs.client import ElevenLabs

## Load Environment Variables

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Instantiate LLM model

### Using ChatGPT 4o-mini

In [3]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_completion_tokens=16384,
                        temperature=0.2)

In [4]:
ollama_model = ChatOllama(model="llama3.2", 
               num_ctx = 16384, 
               num_predict = 16384,
               temperature = 0.2,
               system = 'You are a helpful assistant.')

## Search Tool

In [5]:
# Tavily Search
tavily_search = TavilySearchResults(max_results=2)

# Wikipedia
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Google Trend
google_trend_tool = GoogleTrendsQueryRun(api_wrapper=GoogleTrendsAPIWrapper())

# Google Book
google_book_tool = GoogleBooksQueryRun(api_wrapper=GoogleBooksAPIWrapper())

# Google Places
google_places_tool = GooglePlacesTool()

## Retriever Tool

In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=20)
splitdocs = splitter.split_documents(docs)
embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(documents=splitdocs, embedding=embedding_function)
retriever = vector_store.as_retriever()

In [7]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

## Custom Tool

In [8]:
@tool
def get_stock_closing_price(ticker: str, trading_days: str = '50') -> str:
    ''' 
    Returns the stock's closing prices for the past number of trading days.
    Expects a string of the company's stock ticker and a string 
    for the number of trading days. Return the date and the stock 
    closing prices for the specified number of trading days.
    '''

    n_days = int(trading_days)
    
    alpha_vantage = AlphaVantageAPIWrapper()
    daily_price = alpha_vantage._get_time_series_daily(ticker)
    
    result = ''  
    result += f'Stock Ticker: {ticker}\n\n'
    result += 'Date ' + ' \t       ' + ' Closing Price\n'
    
    counter = 0
    for daily in daily_price['Time Series (Daily)']:
        if counter > n_days:
            break
        else:
            result += daily + '\t' + daily_price['Time Series (Daily)'][daily]['4. close'] + '\n'
            counter += 1
    
    return result

In [9]:
@tool
def earnings_call_transcript(ticker: str = 'MSFT', year: str = '2025', quarter: str = 'Q1') -> str:
    ''' 
    Returns the earnings call transcript for the company ticker
    for the fiscal year and quarter. Expects a string of the company's 
    stock ticker in upper case, a string for the fiscal year in a format like '2025', 
    and a string for the quarter in a format like 'Q1', 'Q2', 'Q3', or 'Q4'. 
    Return the conference call transcript in a string format.
    '''

    tick = ticker.upper()
    advantage_api_key = os.getenv('ALPHAVANTAGE_API_KEY')
    url = f'https://www.alphavantage.co/query?function=EARNINGS_CALL_TRANSCRIPT&symbol={tick}&quarter={year}{quarter}&apikey={advantage_api_key}'
    
    r = requests.get(url)
    data = r.json()

    result = ''
    result += f'Ticker: {data['symbol']}\n'
    result += f'Fiscal Quarter: {data['quarter']}\n\n'
    
    for section in data['transcript']:
        result += f'Speaker: {section['speaker']}\nTitle: {section['title']}\n\n{section['content']}\n\n'
        
    return result

In [24]:
@tool
def get_address_coordinates(address: str) -> str:
    ''' 
    Return the latitude and the longitude corresponding to the
    street address given. If the geocode is not found, it would 
    return the string 'Geocode location  not found'.
    '''
    
    google_maps_api_key = os.getenv('GPLACES_API_KEY')

    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={google_maps_api_key}'

    response = requests.get(url)
    response.raise_for_status()
    geocode_data = response.json()
    if geocode_data['status'] == 'OK':
        latitude = str(geocode_data['results'][0]['geometry']['location']['lat'])
        longitude = str(geocode_data['results'][0]['geometry']['location']['lng'])
        return f'latitude: {latitude}, longitude: {longitude}'
    else:
        return 'Geocode location is not found'

In [25]:
@tool
def haversine(la1: str, lo1: str, la2: str, lo2:str) -> str:
    ''' 
    Return the distance in terms of mile between two locations, 
    both having latitude and the longitude coordinates. The inputs
    are in string format, and the output is a string with the distance
    in miles rounded to two digits.
    '''
    
    # Convert latitude and longitude from degrees to radians
    lat1=float(la1)
    lon1=float(lo1)
    lat2=float(la2)
    lon2=float(lo2)
    
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Radius of Earth in kilometers: 6371 (use 3956 for miles)
    
    r = 3956
    return str(round(c * r, 2))

## Create a list of tools

In [26]:
# tools = load_tools(tool_names=["serpapi", "wolfram-alpha"], llm=chat_model)
# tools = tools + [retriever_tool]

tools_list = load_tools(tool_names=["wolfram-alpha"], llm=ollama_model)
tools_list = tools_list + [tavily_search]
# tools_list = tools_list + [retriever_tool]
tools_list = tools_list + [get_stock_closing_price, earnings_call_transcript, get_address_coordinates, haversine]
tools_list = tools_list + [google_book_tool, google_trend_tool, google_places_tool, wikipedia_tool]

## Prompt Design

In [27]:
today = date.today()
# current_time = str(datetime.now())

In [28]:
def get_current_gps_coordinates():
    g = geocoder.ip('me')#this function is used to find the current information using our IP Add
    if g.latlng is not None: #g.latlng tells if the coordiates are found or not
        return g.latlng
    else:
        return None

In [29]:
latitude, longitude = get_current_gps_coordinates()

In [30]:
print(latitude, longitude)

40.8337 -74.1829


In [31]:
def get_town_state_country():

    g = geocoder.ip('me')
    
    if g.latlng is not None: #g.latlng tells if the coordiates are found or not
        latitude, longitude = g.latlng
    else:
        latitude, longitude = 40.83412, -74.16749

    geolocator=Nominatim(user_agent='geoapiExcises')

    location=geolocator.reverse(str(latitude)+","+str(longitude))
    address=location.raw['address']

    return address['town'], address['state'], address['country']

In [32]:
town, state, country = get_town_state_country()

In [33]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
prompt.messages[0] = SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=f"Your name is Einstein, developed in 2001. You are a helpful assistant. Today is {today}."), additional_kwargs={})


In [35]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

## Create agent

In [36]:
agent = create_tool_calling_agent(llm=chat_model, 
                                  tools=tools_list, 
                                  prompt=prompt)

In [37]:
agent_executor = AgentExecutor(agent=agent, tools=tools_list, verbose=True)

## Define Text_to_Speech function

In [38]:
client = ElevenLabs(api_key=os.getenv('ELEVENLABS_API_KEY'))

def text_to_speech(text, voice="Daniel"):

    voice_catalog = {"Aria": "9BWtsMINqrJLrRacOk9x",
                     "Roger": "CwhRBWXzGAHq8TQ4Fs17",
                     "Sarah": "EXAVITQu4vr4xnSDxMaL",
                     "Laura": "FGY2WhTYpPnrIDTdsKH5",
                     "Charlie": "IKne3meq5aSn9XLyUdCD",
                     "George": "JBFqnCBsd6RMkjVDRZzb",
                     "Callum": "N2lVS1w4EtoT3dr4eOWO",
                     "River": "SAz9YHcvj6GT2YYXdXww",
                     "Liam": "TX3LPaxmHKxFdv7VOQHJ",
                     "Charlotte": "XB0fDUnXU5powFXDhCwa",
                     "Alice": "Xb7hH8MSUJpSbSDYk0k2",
                     "Matilda": "XrExE9yKIg1WjnnlVkGX",
                     "Will": "bIHbv24MWmeRgasZH58o",
                     "Jessica": "cgSgspJ2msm6clMCkdW9",
                     "Eric": "cjVigY5qzO86Huf0OWal",
                     "Chris": "iP95p4xoKVk53GoZ742B",
                     "Brian": "nPczCjzI2devNBz1zQrb",
                     "Daniel": "onwK4e9ZLuTAKqWW03F9",
                     "Lily": "pFZP5JQG7iQjIQuC4Bku",
                     "Bill": "pqHfZKP75CvOlQylNhV4", 
                     "Stacy - Sweet and Cute Chinese": "hkfHEbBvdQFNX4uWHqRF", 
                     "James Gao": "4VZIsMPtgggwNg7OXbPY"}

    if voice in voice_catalog:
        voice_id_code = voice_catalog[voice]
    else:
        voice_id_code = voice_catalog['Daniel']

    audio = client.text_to_speech.convert(text=text, 
                                          voice_id=voice_id_code,
                                          model_id='eleven_turbo_v2_5',
                                          output_format="mp3_44100_64")

    elevenlabs.play(audio)

## Define Chatbot_streaming function

In [39]:
history = []

In [40]:
def chat_stream(message, history):
    
    # messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # text_to_speech(message)
    stream = agent_executor.stream({"input": message, "chat_history": history})

    response = ""
    for chunk in stream:
        if 'output' in chunk:
            response += chunk['output'] or ''
        else:
            continue
        yield response

## Define a Chatbot with voice

In [41]:
def chat_voice(message, history):
    
    # messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = agent_executor.invoke({"input": message, "chat_history": history})
    # text_to_speech(text=response['output'], voice="Daniel")

    return response['output']

## Gradio Interface

In [42]:
chatbot = gr.ChatInterface(fn=chat_stream, type="messages")

In [43]:
chatbot.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.




> Entering new None chain...
Hello, Jim! How can I assist you today?

> Finished chain.


> Entering new None chain...
My name is Einstein. I'm here to help you with any questions or information you need. What can I do for you today?

> Finished chain.


> Entering new None chain...

Invoking: `get_address_coordinates` with `{'address': '101 Darling Ave., Bloomfield, NJ'}`


Your coordinates are approximately:ngitude: -74.1674934
- **Latitude:** 40.8341
- **Longitude:** -74.1675

If you need anything else, feel free to ask!

> Finished chain.


> Entering new None chain...

Invoking: `google_places` with `{'query': "McDonald's near 101 Darling Ave., Bloomfield, NJ"}`


1. McDonald's
Address: 425 Broad St Ctr, Bloomfield, NJ 07003, USA
Google place ID: ChIJcQ7kR2pVwokRG3WWuUZ2lFU
Phone: (973) 429-7244
Website: https://www.mcdonalds.com/us/en-us/location/NJ/BLOOMFIELD/425-BROAD-STREET-CTR/2677.html?cid=RF:YXT:GMB::Clicks


2. McDonald's
Address: 1339 Broad St, Bloomfield, NJ 07003, USA

In [44]:
chatbot.close()

Closing server running on port: 7860
